In [ ]:
import anndata as ad
import scanpy as sc
import numpy as np
import pandas as pd
import copy
import anndata2ri
import logging
import os
import celltypist
from celltypist import models
import scarches as sca
import urllib.request
from scipy.sparse import issparse
from scipy.sparse import csr_matrix
from scipy.stats import median_abs_deviation
import os
import warnings
import liana as li
from liana.method import singlecellsignalr, connectome, cellphonedb, natmi, logfc, cellchat, geometric_mean
import decoupler
from pathlib import Path
from sklearn.cluster import FeatureAgglomeration
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import MinMaxScaler
import math
from cnmf import cNMF
import sys
sys.setrecursionlimit(100000)

warnings.filterwarnings("ignore", category=DeprecationWarning)

import numba
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=180,
    facecolor="white",
    # color_map="YlGnBu",
    frameon=False,
)

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

ro.r('library(scry); library(Biobase); library(NMF); library(scran); library(BiocParallel)')

def gmt_to_decoupler(pth: Path) -> pd.DataFrame:
    """
    Parse a gmt file to a decoupler pathway dataframe.
    """
    from itertools import chain, repeat

    pathways = {}

    with Path(pth).open("r") as f:
        for line in f:
            name, _, *genes = line.strip().split("\t")
            pathways[name] = genes

    return pd.DataFrame.from_records(
        chain.from_iterable(zip(repeat(k), v) for k, v in pathways.items()),
        columns=["geneset", "genesymbol"],
    )

def find_parent(node):
    for pair in tree:
        if node in pair:
            parent = branches[tree.index(pair)]
            if parent in checked_branches and parent==max(branches):
                return
            elif parent in checked_branches:
                find_parent(parent)
            else:
                checked_branches.append(parent)
                global child
                pair = tree[branches.index(parent)]
                for node in pair:
                    if node not in checked_branches and node not in order:
                        child = node
                is_leaf(child)

def is_leaf(child):
    if child in leaves:
        order.append(child)
        find_parent(child)
    else:
        pair = tree[branches.index(child)]
        child = pair[0]
        is_leaf(child)

global checked_branches
global order
global tree
global leaves
global branches

def Gauss(x):
    return 1/(math.sqrt(2*np.pi)) * np.exp(-(x**2)/2)

def KernelDensityEstimater(x, array, h):
    est = 0
    for i in range(len(array)):
        est += Gauss((x - array[i])/h)
    est /= len(array)*h
    return est

#models.download_models(force_update=True, model=["Healthy_Adult_Heart.pkl", "Immune_All_High.pkl",""])
#Immune_All_High = models.Model.load(model="Immune_All_High.pkl")
#Healthy_Adult_Heart = models.Model.load(model="Healthy_Adult_Heart.pkl")

human_datasets = ["GSM4307515","GSM4307516","GSM4307517","GSM4307518","GSM4307519","GSM4307530",
                  "GSM4307531","GSM4307532","GSM4307533","GSM4307534","GSM4307535","GSM4307536",
                  "GSM4307537","GSM4307538","GSM4307539","GSM4307540","GSM4307541","GSM4307542",
                  "GSM4307543","GSM4307544","GSM4307545","GSM4307551","GSM4307552",
                  
                  'GSM4705589','GSM4705590','GSM4705591',
                  
                  "GSM5577199","GSM5577200","GSE131778","GSE145154","GSE155512","GSE196943",
                  
                  "GSM3819856","GSM3819857","GSM3819858","GSM3819859","GSM3819860","GSM3819861",
                  "GSM3819862","GSM3819863",

                  "GSM5905363","GSM5905364","GSM5905365","GSM5905366","GSM5905367","GSM5905368",
                  "GSM5905370","GSM5905371","GSM5905372","GSM5905373","GSM5905375","GSM5905377",
                  
                  "GSE131778", "GSE145154","GSE155512","GSE196943"]

blood_datasets = ["GSM4307519","GSM4307534","GSM4307539","GSM4307544","GSM4307552","GSM5577199",
                  "GSM5577200",
                  
                  "GSM5905363","GSM5905364","GSM5905365","GSM5905366","GSM5905367","GSM5905368",
                  "GSM5905370","GSM5905371","GSM5905372","GSM5905373","GSM5905375","GSM5905377","GSE196943"]

mouse_bone_marrow = ['GSM4762832', 'GSM4762834', 'GSM4762836', 'GSM4762838', 'GSM4762840', 'GSM4762842', 'GSM4762845', 'GSM4762846', 'GSM4762848', 'GSM5792018', 'GSM5792019', 'GSM5792020']

mouse_heart = ['GSM2840136', 'GSM3371744', 'GSM3371745' 'GSM3371746', 'GSM3678221', 'GSM3678222', 'GSM3678223', 'GSM3678224', 'GSM3747856', 'GSM3747857', 'GSM3747858', 'GSM3747859', 'GSM3747860', 'GSM3747861', 'GSM3747862', 'GSM3747863', 'GSM3847600', 'GSM3847601', 'GSM3847602',
'GSM3847603', 'GSM3847604', "GSM4002945", "GSM4002946", "GSM4002947", "GSM4002948", "GSM4002949", 'GSM4005123', 'GSM4005124', 'GSM4005125', 'GSM4005126', 'GSM4005127', 
'GSM4376680', 'GSM4376684', 'GSM4376692', 'GSM4376701', 'GSM4376708', 'GSM4376787', 'GSM4524179', 'GSM4644949', 'GSM4644950', 'GSM4644951', 'GSM4644952', 'GSM4644953', 'GSM4644954', 'GSM4644955', 'GSM4644956',
'GSM4762807', 'GSM4762810', 'GSM4762812', 'GSM4762813', 'GSM4762816', 'GSM4762818', 'GSM4972357', 'GSM4972358', 'GSM4972359', 'GSM4972360', 'GSM4972361', 'GSM4985022', 'GSM4985023', 'GSM4985024', 'GSM4985025', 'GSM5513773', 'GSM5513774', 'GSM5513775', 'GSM5195926', 'GSM5195927'
,'GSM4524179'
]

mouse_peripheral_blood = ['GSM4762820', 'GSM4762821', 'GSM4762823', 'GSM4762825', 'GSM4762827', 'GSM4762829']

mouse_vascular = ['GSM6283570', 'GSM6283571', 'GSM6283572', 'GSM6283573', 'GSM4705592', 'GSM4705593', 'GSM4705594', 'GSM4705595', 'GSM4705596', 'GSM4705597', 'GSM4705598', 'GSM4705599', 'GSM4705600', 'GSM4705601', 'GSM4705602', 'GSM4705603',
'GSM4705604', 'GSM4705605', 'GSM3819834', 'GSM3819835', 'GSM3819836', 'GSM3819837', 'GSM3819838', 'GSM3819839', 'GSM3819840', 'GSM3819841', 'GSM3819842', 'GSM3819843', 'GSM3819844', 'GSM3819845', 'GSM3819846', 'GSM3819847',
'GSM3819848', 'GSM3819849', 'GSM3819850', 'GSM3819851']


In [ ]:
# For each sample, run the following snippet:

adata.obs_names_make_unique()
adata.var_names_make_unique()
if adata.raw==None:
    adata.raw = copy.deepcopy(adata)
adata_copy = (adata)
try:
    if adata.raw.var.columns==["_index"]:
        adata.raw.var.columns=["Genes"]
except:
    pass
adata.layers["counts"] = copy.deepcopy(adata.raw.X)



print("scanpy::Pre-processing...")

adata.layers["normalE4"] = copy.deepcopy(adata.X)
sc.pp.normalize_total(adata, target_sum=10^4, layer="normalE4")
adata.layers["log1p_normalE4"] = copy.deepcopy(adata.layers["normalE4"])
sc.pp.log1p(adata, layer="log1p_normalE4")
adata.layers["log1p"] = copy.deepcopy(adata.X)
sc.pp.log1p(adata, layer="log1p")
adata.layers["counts"] = copy.deepcopy(adata.raw.X)


ro.globalenv["adata"] = adata.layers["log1p_normalE4"].T
ro.r('sce = devianceFeatureSelection(adata)')
binomial_deviance = ro.r("sce").T
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[binomial_deviance.argsort()[-3000:]] = True
adata.var["highly_variable"] = mask

sc.pp.pca(adata, svd_solver="arpack", use_highly_variable=True, n_comps=30)

try:
    sc.tl.tsne(adata, use_rep="SCVI_LATENT_KEY", early_exaggeration=50, n_jobs=22)
    
except:
    sc.tl.tsne(adata, use_rep="X_pca", early_exaggeration=50, n_jobs=22, n_pcs=30)
    
try:
    sc.pp.neighbors(adata, method="gauss", metric="cosine", use_rep="SCVI_LATENT_KEY")
    print("1")
    
except:
    sc.pp.neighbors(adata, method="gauss", metric="cosine", use_rep="X_pca")


sc.tl.umap(adata, min_dist=0.001)
sc.tl.leiden(adata, key_added="groups", resolution=0.4)
sc.tl.diffmap(adata, n_comps=6)
genes_hv3000 = adata.var_names[adata.var["highly_variable"]==True]
try:
    adata.raw.var.columns=["Genes"]
except:
    pass

print("\033[33mNMF...\033[0m")
highly_variable_matrix = adata.layers["log1p"].T[adata.var["highly_variable"]==True].T
ro.globalenv["highly_variable_matrix"] = highly_variable_matrix
ro.r('data <- as.matrix(highly_variable_matrix)')
ro.r('res <- nmf(data, 30, method="snmf/r", seed="nndsvd")')
module_components = ro.r("t(coef(res))")
cell_embeddings = ro.r("basis(res)")
adata.obsm["NMF_embeddings"] = ro.r("basis(res)")


print("\033[33mCelltyping...\033[0m")
try:
    adata_copy.X = adata_copy.X.toarray()
except:
    pass
sc.pp.normalize_per_cell(adata_copy, counts_per_cell_after=10000)
sc.pp.log1p(adata_copy)
if id[:10] in human_datasets:
    if id[:10] in blood_datasets:
        model = models.Model.load(model="Immune_All_High.pkl")
    else:
        model = models.Model.load(model="Healthy_Adult_Heart.pkl")
else:
    if id[:10] in mouse_bone_marrow:
        model = models.Model.load("mouse_bone_marrow.pkl")
    elif id[:10] in mouse_heart:
        model = models.Model.load("mouse_heart_and_peripheral_blood.pkl")
    elif id[:10] in mouse_peripheral_blood:
        model = models.Model.load("mouse_peripheral_blood.pkl")
    elif id[:10] in mouse_vascular:
        model = models.Model.load("mouse_hematopoietic_and_cardiovascular.pkl")

predictions = celltypist.annotate(
    adata_copy, model=model, majority_voting=True
).to_adata()
adata.obs["celltypist_cell_label"] = predictions.obs.loc[
    adata.obs.index, "majority_voting"
]
adata.obs["celltypist_conf_score"] = predictions.obs.loc[
    adata.obs.index, "conf_score"
]

need_sub = [i for i in dict(pd.value_counts(adata.obs["celltypist_cell_label"])) if dict(pd.value_counts(adata.obs["celltypist_cell_label"]))[i] >= 100]
co = 0
for i in range(len(need_sub)):
    co += 1
    if co==1 and len(need_sub)!=1:
        sc.tl.leiden(adata, restrict_to=("celltypist_cell_label", [need_sub[i]]), key_added=need_sub[i], resolution=0.3)
    elif len(need_sub)==1:
        sc.tl.leiden(adata, restrict_to=("celltypist_cell_label", [need_sub[i]]), key_added="cell_types", resolution=0.3)
    elif co!=len(need_sub):
        sc.tl.leiden(adata, restrict_to=(need_sub[i-1], [need_sub[i]]), key_added=need_sub[i], resolution=0.3)
    else:
        sc.tl.leiden(adata, restrict_to=(need_sub[i-1], [need_sub[i]]), key_added="cell_types", resolution=0.3)
if need_sub==[]:
    adata.obs["cell_types"] = adata.obs["celltypist_cell_label"]
cates = list(adata.obs["cell_types"].cat.categories)
for i in range(len(cates)):
    cates.append("_".join(cates[i].split(",")))
cates = pd.Index(cates).unique()
adata.obs["cell_types"] = adata.obs["cell_types"].cat.set_categories(cates)
for i in range(len(adata.obs["cell_types"])):
    adata.obs["cell_types"][i] = "_".join(adata.obs["cell_types"][i].split(","))
adata.obs["cell_types"] = adata.obs["cell_types"].cat.remove_unused_categories()





print("\033[33mDEGs...\033[0m")
adata_deg = adata.copy()
adata_deg.var["which_group_deg"] = "0"
adata_deg.var["load"] = 0
adata_deg.var["logFC"] = 0
adata_deg.var["pval_adj"] = 0
load = 1
deg = []
sc.tl.rank_genes_groups(adata, groupby="cell_types", key_added="deg", method="wilcoxon", use_raw=False)

for i in adata.uns["deg"]["names"]:
    for j in list(i):
        pval_adj = adata.uns["deg"]["pvals_adj"][list(adata.uns["deg"]["names"]).index(i)][list(i).index(j)]
        if adata_deg.var["which_group_deg"][j]=="0" and pval_adj<0.05:
            deg.append(j)
            adata_deg.var["which_group_deg"][j] = list(adata.obs["cell_types"].cat.categories)[list(i).index(j)]
            adata_deg.var["load"][j] = load + (len(deg)-1)//50
            adata_deg.var["logFC"][j] = adata.uns["deg"]["logfoldchanges"][list(adata.uns["deg"]["names"]).index(i)][list(i).index(j)]
            adata_deg.var["pval_adj"][j] = pval_adj
        if len(deg)==300:
            break
    if len(deg)==300:
        break
adata_deg50 = adata_deg[:,adata_deg.var["load"]==1]
adata_deg_append = adata_deg[:,adata_deg.var["load"]>1]
if adata_deg_append.shape[1]!=0:
    APPEND = True
try:
    log1p_deg50 = np.array(adata_deg50.layers["log1p"].T.todense().tolist())
    log1p_deg_append = np.array(adata_deg_append.layers["log1p"].T.todense().tolist())
except:
    log1p_deg50 = np.array(adata_deg50.layers["log1p"].T.tolist())
    log1p_deg_append = np.array(adata_deg_append.layers["log1p"].T.tolist())





print("\033[33mCCI...\033[0m")
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[binomial_deviance.argsort()[-1000:]] = True
adata.var["highly_variable"] = mask
if id[:10] in human_datasets:
    resource_name = 'consensus'
else:
    resource_name = 'mouseconsensus'
    
if len(list(adata.obs["celltypist_cell_label"].cat.categories))==1 or any(adata.obs["celltypist_cell_label"].value_counts()/adata.shape[0]>0.8):
    cellphonedb(adata, use_raw=False, groupby='cell_types', expr_prop=0.1, verbose=False, key_added='cci_major', resource_name=resource_name)
else:
    cellphonedb(adata, use_raw=False, groupby='celltypist_cell_label', expr_prop=0.1, verbose=False, key_added='cci_major', resource_name=resource_name)



print("\033[33mGSEA...\033[0m")
if len(list(adata.obs["celltypist_cell_label"].cat.categories))==1 or any(adata.obs["celltypist_cell_label"].value_counts()/adata.shape[0]>0.8):
    cell_types = list(adata.obs["cell_types"].unique())
else:
    cell_types = list(adata.obs["celltypist_cell_label"].unique())
if id[:10] in human_datasets:
    reactome = gmt_to_decoupler("c2.all.v2023.2.Hs.symbols.gmt")
else:
    reactome = gmt_to_decoupler("m2.all.v2023.2.Mm.symbols.gmt")
decoupler.run_aucell(
    adata,
    reactome,
    source="geneset",
    target="genesymbol",
    use_raw=False,
)


pathway_activeness = pd.DataFrame(adata.obsm["aucell_estimate"], columns = adata.obsm["aucell_estimate"].columns)
scaler = MinMaxScaler()
pathway_activeness = pd.DataFrame(scaler.fit_transform(pathway_activeness), index=pathway_activeness.index, columns = adata.obsm["aucell_estimate"].columns)
pathway_activeness_cluster = pd.Series()
#if len(cell_types)!=1:
if len(list(adata.obs["celltypist_cell_label"].cat.categories))==1 or any(adata.obs["celltypist_cell_label"].value_counts()/adata.shape[0]>0.8):
    for cell_type in cell_types:
        pathway_activeness_cluster = pd.concat([pathway_activeness_cluster,
                                                pd.DataFrame(pathway_activeness, index = adata.obs_names[adata.obs["cell_types"]==cell_type]).mean(axis=0).round(decimals=3)],
                                            axis=1,
                                            ignore_index=False)
else:
    for cell_type in cell_types:
        pathway_activeness_cluster = pd.concat([pathway_activeness_cluster,
                                                pd.DataFrame(pathway_activeness, index = adata.obs_names[adata.obs["celltypist_cell_label"]==cell_type]).mean(axis=0).round(decimals=3)],
                                            axis=1,
                                            ignore_index=False)
pathway_activeness_cluster.columns = [0]+cell_types
pathway_activeness_cluster = pd.DataFrame(pathway_activeness_cluster, index = adata.obsm["aucell_estimate"].columns, columns = cell_types)
DEP = (pathway_activeness_cluster.std(axis=1).sort_values(ascending=False)[:1000]).index
pathway_activeness_cluster = pathway_activeness_cluster.loc[DEP]

agglo = AgglomerativeClustering()
agglo.fit(pathway_activeness_cluster)
branches = list(range(len(pathway_activeness_cluster), len(pathway_activeness_cluster)+len(agglo.children_)))
leaves = list(range(len(pathway_activeness_cluster)))
tree = [list(i) for i in agglo.children_]
checked_branches = [branches[0]]
order = list(agglo.children_[0])
branch = branches[0]
find_parent(branch)
order_pathways = order
hier_pathways = [DEP[i] for i in order_pathways]

agglo = AgglomerativeClustering()
agglo.fit(pathway_activeness_cluster.T)
branches = list(range(len(cell_types), len(cell_types)+len(agglo.children_)))
leaves = list(range(len(cell_types)))
tree = [list(i) for i in agglo.children_]
checked_branches = [branches[0]]
order = list(agglo.children_[0])
branch = branches[0]
find_parent(branch)
order_cell_types = order
hier_cell_types = [cell_types[i] for i in order]

pathway_activeness_cluster = pd.DataFrame(pathway_activeness_cluster, index=hier_pathways, columns=hier_cell_types)
new_pathway_names = []
for i in range(len(pathway_activeness_cluster.index)):
    new_pathway_names.append(" ".join(pathway_activeness_cluster.index[i].split("_")))
pathway_activeness_cluster.index = new_pathway_names

adata.obsm["aucell_estimate"] = adata.obsm["aucell_estimate"][DEP]

print("\033[32mSaving...\033[0m")
basic = np.concatenate(
    (np.array(adata.obs_names)[:,None],    # cell barcode
    np.array(adata.obs["celltypist_cell_label"])[:,None],    # leiden group numbers
    np.multiply(adata.obsm["X_tsne"],100).astype(int),    # tsne (x,y) => (100x,100y)
    np.multiply(adata.obsm["X_umap"],100).astype(int),    # umap (x,y) => (100x,100y)
    np.multiply(adata.obsm["X_diffmap"],100000).astype(int),    # diffusion components
    np.array(adata.obs["cell_types"])[:,None],    # celltypist cell type
    ), axis=1)
id = id[:10]
np.savetxt(file_path, basic, delimiter=",", fmt="%s", encoding = "utf-8")
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["cell,cluster,snex,sney,umapx,umapy,diff_1,diff_2,diff_3,diff_4,diff_5,diff_6,cell_type\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))


# DEG expressions
table = np.concatenate(
    (np.array(adata.obs_names)[:,None],    # cell barcode
    np.round(log1p_deg50.T,decimals=2),    # log1p data
    ), axis=1)
np.savetxt(file_path, table, delimiter=",", fmt="%s", encoding = "utf-8")
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["cell,"+",".join([i for i in adata_deg50.var_names.to_list()])+"\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))

table = np.concatenate(
    (np.array(adata.obs_names)[:,None],    # cell barcode
    np.round(log1p_deg_append.T,decimals=2),    # log1p data
    ), axis=1)
np.savetxt(file_path, table, delimiter=",", fmt="%s", encoding = "utf-8")
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["cell,"+",".join([i for i in adata_deg_append.var_names.to_list()])+"\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))
    
# NME embeddings
table = np.concatenate(
    (np.array(adata.obs_names)[:,None],    # cell barcode
    np.round(cell_embeddings,decimals=3),    # programs expression level
    ), axis=1)
np.savetxt(file_path, table, delimiter=",", fmt="%s", encoding = "utf-8")
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["cell,"+",".join(["nmf_"+str(j) for j in range(1,k_select+1)])+"\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))


# DEG info
hv_info = np.concatenate((
    np.array(adata_deg.var_names)[[list(adata_deg.var_names).index(i) for i in deg]][:,None],
    np.array(adata_deg.var["which_group_deg"])[[list(adata_deg.var_names).index(i) for i in deg]][:,None],
    np.array(adata_deg.var["logFC"])[[list(adata_deg.var_names).index(i) for i in deg]][:,None],
    np.array(adata_deg.var["pval_adj"])[[list(adata_deg.var_names).index(i) for i in deg]][:,None],
), axis=1)
np.savetxt(file_path, hv_info, delimiter=",", fmt="%s", encoding = "utf-8")
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["gene,cell_type,logFC,pval_adj\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))


# CCI
adata.uns["cci_major"][adata.uns["cci_major"]["cellphone_pvals"]<=0.05].to_csv(file_path, encoding="utf-8", header=False)
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["lr_id,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means,cellphone_pvals\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))


# NMF top genes
gene_programs = np.array(module_components, dtype=object)
np.savetxt(file_path, gene_programs, delimiter=",", fmt="%s")
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["program_id,"+",".join(["gene_"+str(j) for j in range(1,51)])+"\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))



pathway_activeness_cluster.to_csv(file_path, encoding="utf-8", index=True, header=False)
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["pathway,"+",".join(hier_cell_types)+"\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))


table_number_in_db = 1226
with open(file_path, "a") as tmp:
    tmp.write(
        "ALTER TABLE `TABLE "+str(table_number_in_db+pos*8+1)+"` RENAME `"+id+"_id`;\n"+
        "ALTER TABLE `TABLE "+str(table_number_in_db+pos*8+2)+"` RENAME `"+id+"_50`;\n"+
        "ALTER TABLE `TABLE "+str(table_number_in_db+pos*8+3)+"` RENAME `"+id+"_app`;\n"+
        "ALTER TABLE `TABLE "+str(table_number_in_db+pos*8+4)+"` RENAME `"+id+"_cci`;\n"+
        "ALTER TABLE `TABLE "+str(table_number_in_db+pos*8+5)+"` RENAME `"+id+"_hv`;\n"+
        "ALTER TABLE `TABLE "+str(table_number_in_db+pos*8+6)+"` RENAME `"+id+"_module`;\n"+
        "ALTER TABLE `TABLE "+str(table_number_in_db+pos*8+7)+"` RENAME `"+id+"_pathway`;\n"+
        "ALTER TABLE `TABLE "+str(table_number_in_db+pos*8+8)+"` RENAME `"+id+"_nmf`;\n\n"
    )


In [ ]:
# for each dataset, run the following snippet:

adata.obs_names_make_unique()
adata.var_names_make_unique()
adata.raw = copy.deepcopy(adata)
adata_copy = copy.deepcopy(adata)
adata.layers["counts"] = copy.deepcopy(adata.raw.X)

print("scanpy::Pre-processing...")
adata.layers["normalE4"] = copy.deepcopy(adata.X)
sc.pp.normalize_total(adata, target_sum=10^4, layer="normalE4")
adata.layers["log1p_normalE4"] = copy.deepcopy(adata.layers["normalE4"])
sc.pp.log1p(adata, layer="log1p_normalE4")
adata.layers["log1p"] = copy.deepcopy(adata.X)
sc.pp.log1p(adata, layer="log1p")
ro.globalenv["adata"] = adata.layers["log1p_normalE4"].T
ro.r('sce = devianceFeatureSelection(adata)')
binomial_deviance = ro.r("sce").T
mask = np.zeros(adata.var_names.shape, dtype=bool)
mask[binomial_deviance.argsort()[-3000:]] = True
adata.var["highly_variable"] = mask
sc.pp.pca(adata, svd_solver="arpack", use_highly_variable=True, n_comps=30)
sc.tl.tsne(adata, use_rep="SCVI_LATENT_KEY", early_exaggeration=50, n_jobs=22)
sc.pp.neighbors(adata, method="gauss", metric="cosine", use_rep="SCVI_LATENT_KEY")
sc.tl.umap(adata, min_dist=0.001)
sc.tl.diffmap(adata, n_comps=6)
genes_hv3000 = adata.var_names[adata.var["highly_variable"]==True]

print("\033[33mCelltyping...\033[0m")
try:
    adata_copy.X = adata_copy.X.toarray()
except:
    pass
sc.pp.normalize_per_cell(adata_copy, counts_per_cell_after=10000)
sc.pp.log1p(adata_copy)
if id[:9] in human_datasets:
    if id[:9] in blood_datasets:
        model = models.Model.load(model="Immune_All_High.pkl")
    else:
        model = models.Model.load(model="Healthy_Adult_Heart.pkl")
else:
    if id[:9] in ["GSE193316"]:
        model = models.Model.load("mouse_bone_marrow.pkl")
    elif id[:9] in ["GSE119355","GSE128509","GSE132144","GSE146285","GSE163129","GSE169332","GSE135296","GSE135310","GSE163465"]:
        model = models.Model.load("mouse_heart_and_peripheral_blood.pkl")
    elif id[:9] in ["GSE184073","GSE131776","GSE157244","GSE207275"]:
        model = models.Model.load("mouse_hematopoietic_and_cardiovascular.pkl")
    elif id[:9] in ["GSE130699","GSE153480"]:
        model = models.Model.load("mouse_post_natal.pkl")

predictions = celltypist.annotate(
    adata_copy, model=model, majority_voting=True
).to_adata()
adata.obs["celltypist_cell_label"] = predictions.obs.loc[
    adata.obs.index, "majority_voting"
]
adata.obs["celltypist_conf_score"] = predictions.obs.loc[
    adata.obs.index, "conf_score"
]

print("\033[33mDEG...\033[0m")
adata_deg = adata.copy()
adata_deg.var["which_group_deg"] = "0"
adata_deg.var["load"] = 0
adata_deg.var["logFC"] = 0
adata_deg.var["pval_adj"] = 0
load = 1
deg = []
tot_num_DEG = len(adata.obs["batch"].cat.categories)*50
sc.tl.rank_genes_groups(adata, groupby="batch", key_added="deg", method="wilcoxon", use_raw=False)
for i in adata.uns["deg"]["names"]:
    for j in list(i):
        pval_adj = adata.uns["deg"]["pvals_adj"][list(adata.uns["deg"]["names"]).index(i)][list(i).index(j)]
        if adata_deg.var["which_group_deg"][j]=="0" and pval_adj<0.05:
            deg.append(j)
            adata_deg.var["which_group_deg"][j] = list(adata.obs["batch"].cat.categories)[list(i).index(j)]
            adata_deg.var["load"][j] = load + (len(deg)-1)//50
            adata_deg.var["logFC"][j] = adata.uns["deg"]["logfoldchanges"][list(adata.uns["deg"]["names"]).index(i)][list(i).index(j)]
            adata_deg.var["pval_adj"][j] = pval_adj
        if len(deg)==tot_num_DEG:
            break
    if len(deg)==tot_num_DEG:
        break


print("\033[33mGSEA...\033[0m")
cell_types = list(adata.obs["batch"].unique())
if id[:9] in ["GSE184073","GSE196943","GSE155512","GSE145154","GSE131778"]:
    reactome = gmt_to_decoupler("c2.all.v2023.2.Hs.symbols.gmt")
else:
    reactome = gmt_to_decoupler("m2.all.v2023.2.Mm.symbols.gmt")
decoupler.run_aucell(
    adata,
    reactome,
    source="geneset",
    target="genesymbol",
    use_raw=False,
)


pathway_activeness = pd.DataFrame(adata.obsm["aucell_estimate"], columns = adata.obsm["aucell_estimate"].columns)
scaler = MinMaxScaler()
pathway_activeness = pd.DataFrame(scaler.fit_transform(pathway_activeness), index=pathway_activeness.index, columns = adata.obsm["aucell_estimate"].columns)
pathway_activeness_cluster = pd.Series()
for cell_type in cell_types:
    pathway_activeness_cluster = pd.concat([pathway_activeness_cluster,
                                            pd.DataFrame(pathway_activeness, index = adata.obs_names[adata.obs["batch"]==cell_type]).mean(axis=0).round(decimals=3)],
                                        axis=1,
                                        ignore_index=False)
pathway_activeness_cluster.columns = [0]+cell_types
pathway_activeness_cluster = pd.DataFrame(pathway_activeness_cluster, index = adata.obsm["aucell_estimate"].columns, columns = cell_types)
DEP = (pathway_activeness_cluster.std(axis=1).sort_values(ascending=False)[:1000]).index
pathway_activeness_cluster = pathway_activeness_cluster.loc[DEP]

agglo = AgglomerativeClustering()
agglo.fit(pathway_activeness_cluster)
branches = list(range(len(pathway_activeness_cluster), len(pathway_activeness_cluster)+len(agglo.children_)))
leaves = list(range(len(pathway_activeness_cluster)))
tree = [list(i) for i in agglo.children_]
checked_branches = [branches[0]]
order = list(agglo.children_[0])
branch = branches[0]
find_parent(branch)
order_pathways = order
hier_pathways = [DEP[i] for i in order_pathways]

agglo = AgglomerativeClustering()
agglo.fit(pathway_activeness_cluster.T)
branches = list(range(len(cell_types), len(cell_types)+len(agglo.children_)))
leaves = list(range(len(cell_types)))
tree = [list(i) for i in agglo.children_]
checked_branches = [branches[0]]
order = list(agglo.children_[0])
branch = branches[0]
find_parent(branch)
order_cell_types = order
hier_cell_types = [cell_types[i] for i in order]

pathway_activeness_cluster = pd.DataFrame(pathway_activeness_cluster, index=hier_pathways, columns=hier_cell_types)
new_pathway_names = []
for i in range(len(pathway_activeness_cluster.index)):
    new_pathway_names.append(" ".join(pathway_activeness_cluster.index[i].split("_")))
pathway_activeness_cluster.index = new_pathway_names

adata.obsm["aucell_estimate"] = adata.obsm["aucell_estimate"][DEP]

print("\033[32mSaving...\033[0m")
basic = np.concatenate(
    (np.array(adata.obs_names)[:,None],    # cell barcode
    np.array(adata.obs["batch"])[:,None],    # leiden group numbers
    np.multiply(adata.obsm["X_tsne"],100).astype(int),    # tsne (x,y) => (100x,100y)
    np.multiply(adata.obsm["X_umap"],100).astype(int),    # umap (x,y) => (100x,100y)
    np.multiply(adata.obsm["X_diffmap"],100000).astype(int),    # diffusion components
    np.array(adata.obs["celltypist_cell_label"])[:,None],    # celltypist cell type
    ), axis=1)
id = id[:9]
np.savetxt(file_path, basic, delimiter=",", fmt="%s", encoding = "utf-8")
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["cell,batch,snex,sney,umapx,umapy,diff_1,diff_2,diff_3,diff_4,diff_5,diff_6,cell_type\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))



hv_info = np.concatenate((
    np.array(adata_deg.var_names)[[list(adata_deg.var_names).index(i) for i in deg]][:,None],
    np.array(adata_deg.var["which_group_deg"])[[list(adata_deg.var_names).index(i) for i in deg]][:,None],
    np.array(adata_deg.var["logFC"])[[list(adata_deg.var_names).index(i) for i in deg]][:,None],
    np.array(adata_deg.var["pval_adj"])[[list(adata_deg.var_names).index(i) for i in deg]][:,None],
), axis=1)
np.savetxt(file_path, hv_info, delimiter=",", fmt="%s", encoding = "utf-8")
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["gene,cell_type,logFC,pval_adj\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))



pathway_activeness_cluster.to_csv(file_path, encoding="utf-8", index=True, header=False)
with open(file_path, "r") as tmp:
    temp = tmp.readlines()
    temp = ["pathway,"+",".join(hier_cell_types)+"\n"]+temp
with open(file_path, "w") as tmp:
    tmp.write("".join(temp))
